In [102]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/student-shopee-code-league-marketing-analytics/train.csv
/kaggle/input/student-shopee-code-league-marketing-analytics/sample_submission_0_1.csv
/kaggle/input/student-shopee-code-league-marketing-analytics/test.csv
/kaggle/input/student-shopee-code-league-marketing-analytics/users.csv


In [103]:
df = pd.read_csv('../input/student-shopee-code-league-marketing-analytics/train.csv')
df_test = pd.read_csv('../input/student-shopee-code-league-marketing-analytics/test.csv')
len(df)

73539

In [104]:
from sklearn.utils import resample

# separate minority and majority classes
not_opened = df[df.open_flag==0]
opened = df[df.open_flag==1]

# upsample minority
not_opened_downsample = resample(not_opened,
                          replace=False, # sample with replacement
                          n_samples=len(opened)) # reproducible results

# combine majority and upsampled minority
df = pd.concat([opened, not_opened_downsample])

In [105]:
len(df)

22912

In [106]:
df = df.drop('row_id', axis=1)
df = df.drop('grass_date', axis=1)

df_test = df_test.drop('row_id', axis=1)
df_test = df_test.drop('grass_date', axis=1)

In [107]:
userdf = pd.read_csv('../input/student-shopee-code-league-marketing-analytics/users.csv')

In [108]:
df = pd.merge(df, userdf, on='user_id', how='inner')
df_test = pd.merge(df_test, userdf, on='user_id', how='inner')

In [109]:
df = df.drop('user_id', axis=1)
df_test = df_test.drop('user_id', axis=1)

In [110]:
df['attr_1'] = df['attr_1'].fillna(df['attr_1'].mean())
df['attr_2'] = df['attr_2'].fillna(df['attr_2'].mean())
df['attr_3'] = df['attr_3'].fillna(df['attr_3'].mean())
df['age'] = df['age'].fillna(df['age'].mean())

df_test['attr_1'] = df_test['attr_1'].fillna(df_test['attr_1'].mean())
df_test['attr_2'] = df_test['attr_2'].fillna(df_test['attr_2'].mean())
df_test['attr_3'] = df_test['attr_3'].fillna(df_test['attr_3'].mean())
df_test['age'] = df_test['age'].fillna(df_test['age'].mean())

In [111]:
labelencoder= LabelEncoder()
df['domain'] = labelencoder.fit_transform(df['domain'])
df_test['domain'] = labelencoder.fit_transform(df_test['domain'])

In [112]:
df = df.replace('Never open', 0)
df = df.replace('Never login', 0)
df = df.replace('Never checkout', 0)

df_test = df_test.replace('Never open', 0)
df_test = df_test.replace('Never login', 0)
df_test = df_test.replace('Never checkout', 0)

In [113]:
df['last_open_day'] = df['last_open_day'].astype(int)
df['last_login_day'] = df['last_login_day'].astype(int)
df['last_checkout_day'] = df['last_checkout_day'].astype(int)


df_test['last_open_day'] = df_test['last_open_day'].astype(int)
df_test['last_login_day'] = df_test['last_login_day'].astype(int)
df_test['last_checkout_day'] = df_test['last_checkout_day'].astype(int)

In [114]:
min_max_scaler = MinMaxScaler()
col_name = df.columns
col_test_name = df_test.columns

# Normalizing x
x = pd.DataFrame(data = min_max_scaler.fit_transform(df), columns = col_name)
y = pd.DataFrame(data = min_max_scaler.fit_transform(df_test), columns = col_test_name)

In [115]:
from sklearn.model_selection import train_test_split

train_features = x.drop('open_flag', axis = 1)
train_target = x['open_flag']
# train_features, test_features, train_target, test_target = train_test_split(features, target, test_size = 0.2, random_state = 5)
# print("Train features shape : ", train_features.shape)
# print("Train target shape   : ", train_target.shape)
# print("Test features shape  : ", test_features.shape)
# print("Test target shape    : ", test_target.shape)

In [116]:
train_target = pd.get_dummies(train_target)

In [98]:
train_target

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
22907    0.0
22908    0.0
22909    0.0
22910    0.0
22911    0.0
Name: open_flag, Length: 22912, dtype: float64

In [117]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, input_dim=19,activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 64)                1280      
_________________________________________________________________
dense_20 (Dense)             (None, 512)               33280     
_________________________________________________________________
dense_21 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_23 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_24 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_25 (Dense)             (None, 16)               

In [118]:
import keras.backend as K
def matthews_correlation(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def mcc_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0) * 1e2
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0) / 1e2
    
    up = tp*tn - fp*fn
    down = K.sqrt((tp+fp) * (tp+fn) * (tn+fp) * (tn+fn))
    
    mcc = up / (down + K.epsilon())
    mcc = tf.where(tf.math.is_nan(mcc), tf.zeros_like(mcc), mcc)
    
    return 1 - K.mean(mcc)

adam = tf.keras.optimizers.Adam(0.0001)
# sgd = tf.keras.optimizers.SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=adam,
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics = ['accuracy', matthews_correlation])


# model.compile(loss='mean_squared_error', optimizer=opt)

In [ ]:
history = model.fit(train_features, train_target, epochs=100, batch_size=8,  verbose=1, validation_split=0.1)

Epoch 1/100
2578/2578 [==============================] - 9s 4ms/step - loss: 0.5804 - accuracy: 0.7317 - matthews_correlation: 0.4486 - val_loss: 0.5198 - val_accuracy: 0.7613 - val_matthews_correlation: 0.5262
Epoch 2/100
2578/2578 [==============================] - 9s 3ms/step - loss: 0.5001 - accuracy: 0.7681 - matthews_correlation: 0.5348 - val_loss: 0.5897 - val_accuracy: 0.7352 - val_matthews_correlation: 0.4740
Epoch 3/100
2578/2578 [==============================] - 9s 3ms/step - loss: 0.4987 - accuracy: 0.7710 - matthews_correlation: 0.5410 - val_loss: 0.4411 - val_accuracy: 0.8259 - val_matthews_correlation: 0.6460
Epoch 4/100
2578/2578 [==============================] - 9s 3ms/step - loss: 0.5010 - accuracy: 0.7753 - matthews_correlation: 0.5414 - val_loss: 0.5307 - val_accuracy: 0.7967 - val_matthews_correlation: 0.5977
Epoch 5/100
2578/2578 [==============================] - 9s 3ms/step - loss: 0.5004 - accuracy: 0.7744 - matthews_correlation: 0.5427 - val_loss: 0.5410 - v

In [73]:
pred = model.predict(y)

In [74]:
pred[0]

array([0.22483163, 0.7751683 ], dtype=float32)

In [75]:
list3 = []
for i in pred:
    list3.append(np.argmax(i))
predicted = np.array(list3)
print(predicted)

[1 1 0 ... 0 1 1]


In [76]:
final_pred = pd.DataFrame(predicted, columns= ['open_flag'])

In [77]:
final_pred.index.names = ['row_id']

In [78]:
final_pred

,open_flag
row_id,
0,1
1,1
2,0
3,0
4,0
...,...
55965,0
55966,0
55967,0


In [79]:
final_pred.to_csv('output.csv')